---
* Title:  Feature Importance Ensemble
* Author: Divish Rengasamy
* Date:   14 May 2020
---

# Dependencies

* eli5 version: 0.10.1
* matplotlib version: 3.1.3
* numpy version: 1.18.2
* pandas version: 1.0.3
* plotly version: 4.7.1
* shap version: 0.35.0
* torch version: 1.4.0
* scipy version: 1.4.1
* sklearn version: 0.22
* tensorflow_addons version: 0.9.1
* captum version: 0.2.0

In [8]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly as py
import shap
from scipy import sparse
from sklearn.datasets import make_regression
from ensembleFI import ensemble_feature_importance
plt.style.use("seaborn-pastel")
import timeit
%matplotlib inline

# Generate Dataset 

In [9]:
_noise = np.arange(0,6,2)
_noise

array([0, 2, 4])

In [10]:
_informative = np.arange(20,120,20)
_informative

array([ 20,  40,  60,  80, 100])

In [11]:
_num_feats = np.arange(20,120, 40)
_num_feats

array([ 20,  60, 100])

In [12]:
_n_samples = np.arange(1000,6000,2000)
_n_samples

array([1000, 3000, 5000])

In [13]:
noise_level = list()
informative_level = list()
num_features_level = list()
for i in range(len(_noise)):
    for j in range(len(_informative)):
        for k in range(len(_num_feats)):
            noise_level.append(_noise[i])
            informative_level.append(_informative[j])
            num_features_level.append(_num_feats[k])

In [15]:
# TODO: 
#COMPARE model error vs importance error
# experimental variable: Noise, informative, number of samples, effective rank, number of features

#IMPORTANT
##################################################
# NORMALIZE RF+GB+DNN SV THEN RECALCULATE ERROR
# rank + mean (done)
##################################################

In [ ]:
#num_samples = 1000
rf_mae = list()
gb_mae = list()
dnn_mae = list()

err_rf_pi = list()
err_gb_pi = list()
err_dnn_pi = list()
err_rf_sv = list() 
err_gb_sv = list()
err_dnn_sv = list()
err_dnn_ig = list()

err_mean = list()
err_median = list()
err_mode = list()
err_box = list()
err_tau = list()
err_major = list()
err_kendall = list()
err_spearman = list()
non_sig_ratio = list()

num_sam = 1000
for i in range(len(_noise)):
    for j in range(len(_informative)):
        for k in range(len(_num_feats)):
            print(f"Noise:{_noise[i]}, n_inform:{_informative[j]}%, n_feats:{_num_feats[k]}, n_samples:1000")
            start = timeit.default_timer()

            # high informative high noise
            _features, _output, _coef = make_regression(
                n_samples=num_sam,
                # three features
                n_features=_num_feats[k],
                # where only two features are useful,
                n_informative=int(_num_feats[k]*(_informative[j]/100)),
                # a single target value per observation
                n_targets=1,
                # 0.0 standard deviation of the guassian noise
                noise=_noise[i],
                # show the true coefficient used to generated the data
                coef=True,
            )

            temp_non_sig_ratio ,temp_rf_mae, temp_gb_mae, temp_dnn_mae, temp_err_rf_pi, temp_err_gb_pi, temp_err_dnn_pi, temp_err_rf_sv, temp_err_gb_sv, temp_err_dnn_sv, temp_err_dnn_ig, temp_err_mean, temp_err_median, temp_err_mode, temp_err_box, temp_err_tau, temp_err_major, temp_err_kendall, temp_err_spearman = ensemble_feature_importance(_num_feats[k], _features, _output, _coef, informative=int((_num_feats[k])*(_informative[j]/100)), noise=_noise[i])

            stop = timeit.default_timer()

            print('Time: ', stop - start)
            non_sig_ratio.append(temp_non_sig_ratio)
            rf_mae.append(temp_rf_mae)
            gb_mae.append(temp_gb_mae)
            dnn_mae.append(temp_dnn_mae)
            err_rf_pi.append(temp_err_rf_pi)
            err_gb_pi.append(temp_err_gb_pi)
            err_dnn_pi.append(temp_err_dnn_pi)
            err_rf_sv.append(temp_err_rf_sv)
            err_gb_sv.append(temp_err_gb_sv)
            err_dnn_sv.append(temp_err_dnn_sv)
            err_dnn_ig.append(temp_err_dnn_ig)
            err_mean.append(temp_err_mean)
            err_median.append(temp_err_median)
            err_mode.append(temp_err_mode)
            err_box.append(temp_err_box)
            err_tau.append(temp_err_tau)
            err_major.append(temp_err_major)
            err_kendall.append(temp_err_kendall)
            err_spearman.append(temp_err_spearman)
            

Noise:0, n_inform:20%, n_feats:20, n_samples:1000
predicted_shape:(200,) y_test shape:(200,)
rf_mae:40.61946632827824
predicted_shape:(200,) y_test shape:(200,)
gb_mae:32.62023238805669


Setting feature_perturbation = "tree_path_dependent" because no background data was given.
The sklearn.ensemble.gradient_boosting module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.


Epoch 1/1000 => Loss: 19623.107422
Epoch 101/1000 => Loss: 12228.665039
Epoch 201/1000 => Loss: 6477.524902
Epoch 301/1000 => Loss: 3282.226562
Epoch 401/1000 => Loss: 1573.626221
Epoch 501/1000 => Loss: 743.093140
Epoch 601/1000 => Loss: 319.109619
Epoch 701/1000 => Loss: 141.048431
Epoch 801/1000 => Loss: 75.376892
Epoch 901/1000 => Loss: 22.293425
predicted_shape:(200, 1) y_test shape:(200, 1)
dnn_mae: 39.82363135901057
Instructions for updating:
If using Keras pass *_constraint arguments to layers.



RF PI error: 0.03005494686353234
GB PI error: 0.030634981907259197
DNN PI error: 0.2945860694439427
RF+GB+DNN PI error: 0.11842533273824477

RF SV error: 0.02094414894025356
GB SV error: 0.012606987552258805
DNN SV error: 0.22789883713942496
RF+GB+DNN SV error: 0.013210644492654014

GB PI error: 0.030634981907259197
GB SV error: 0.012606987552258805
GB PI + GB SV error: 0.021620984729758992

RF PI error: 0.03005494686353234
RF SV error: 0.02094414894025356
RF PI + RF SV error: 0.02549954790189295

DNN PI error: 0.2945860694439427
DNN SV error: 0.0060807969854496675
DNN IG error: 0.22789883713942496
DNN PI + DNN SV + DNN IG error: 0.1676239981982311

Dummy Average of 0.0 error: 0.14526220763881112
Dummy Average of 0.5 error: 0.44763636974754323
Dummy Average of 0.1 error: 0.8547377923611889

 All MAE (mean): 0.07222526738495233
 All MAE (median): 0.02240805866386396
 All MAE (mode): 0.07222526458226339
 All MAE (box-whiskers): 0.042788056024543365
 All MAE (tau-test): 0.036706003724693

Time:  474.21831193205435
Noise:0, n_inform:20%, n_feats:60, n_samples:1000
predicted_shape:(200,) y_test shape:(200,)
rf_mae:92.53724171202168
predicted_shape:(200,) y_test shape:(200,)
gb_mae:86.71708327084536


...

Epoch 1/1000 => Loss: 158898.046875
Epoch 101/1000 => Loss: 120683.171875
Epoch 201/1000 => Loss: 93635.820312
Epoch 301/1000 => Loss: 71752.500000
Epoch 401/1000 => Loss: 54443.554688
Epoch 501/1000 => Loss: 41012.003906
Epoch 601/1000 => Loss: 30854.287109
Epoch 701/1000 => Loss: 22822.957031
Epoch 801/1000 => Loss: 16512.814453
Epoch 901/1000 => Loss: 12085.656250
predicted_shape:(200, 1) y_test shape:(200, 1)
dnn_mae: 100.25545145004348



RF PI error: 0.29947640732294645
GB PI error: 0.25513152783326076
DNN PI error: 0.3437525886432266
RF+GB+DNN PI error: 0.27755358364856675

RF SV error: 0.2909446786093145
GB SV error: 0.2516615576104236
DNN SV error: 0.34302981204114774
RF+GB+DNN SV error: 0.21761161511724642

GB PI error: 0.25513152783326076
GB SV error: 0.2516615576104236
GB PI + GB SV error: 0.25210583202702425

RF PI error: 0.29947640732294645
RF SV error: 0.2909446786093145
RF PI + RF SV error: 0.2951012530153211

DNN PI error: 0.3437525886432266
DNN SV error: 0.14904217012392287
DNN IG error: 0.34302981204114774
DNN PI + DNN SV + DNN IG error: 0.24249798037160178

Dummy Average of 0.0 error: 0.388195078494882
Dummy Average of 0.5 error: 0.3041113592893298
Dummy Average of 0.1 error: 0.611804921505118

 All MAE (mean): 0.2103817555486717
 All MAE (median): 0.26879271513101927
 All MAE (mode): 0.21038174952633826
 All MAE (box-whiskers): 0.22919466973048616
 All MAE (tau-test): 0.2395488892452909
 All MAE (majori

Time:  566.048215304967
Noise:2, n_inform:80%, n_feats:100, n_samples:1000
predicted_shape:(200,) y_test shape:(200,)
rf_mae:362.62996181074186
predicted_shape:(200,) y_test shape:(200,)
gb_mae:334.66300315492396


Setting feature_perturbation = "tree_path_dependent" because no background data was given.


Epoch 1/1000 => Loss: 251234.515625
Epoch 101/1000 => Loss: 219817.859375
Epoch 201/1000 => Loss: 186241.281250
Epoch 301/1000 => Loss: 159385.750000
Epoch 401/1000 => Loss: 124893.156250
Epoch 501/1000 => Loss: 100920.507812
Epoch 601/1000 => Loss: 78797.656250
Epoch 701/1000 => Loss: 62146.320312
Epoch 801/1000 => Loss: 49503.808594
Epoch 901/1000 => Loss: 39174.996094
predicted_shape:(200, 1) y_test shape:(200, 1)
dnn_mae: 196.4222237261838


In [ ]:
d_1 = {'non_sig_ratio':non_sig_ratio,'noise_level':noise_level, 'informative_level':informative_level,'num_features_level':num_features_level,
     'non_sig_ratio':non_sig_ratio,'rf_mae':rf_mae,'gb_mae':gb_mae,'dnn_mae':dnn_mae,'err_rf_pi':err_rf_pi,
     'err_gb_pi':err_gb_pi,'err_dnn_pi':err_dnn_pi,'err_rf_sv':err_rf_sv,'err_gb_sv':err_gb_sv,
     'err_dnn_sv':err_dnn_sv,'err_dnn_ig':err_dnn_ig,'err_mean':err_mean,'err_median':err_median,
     'err_mode':err_mode,'err_box':err_box,'err_tau':err_tau,'err_major':err_major,'err_kendall':err_kendall,
     'err_spearman':err_spearman}

df_1000_samples_rank_avg_1 = pd.DataFrame(data=d_1)
df_1000_samples_rank_avg_1.to_csv('df_1000_samples_rank_avg_1.csv', index=False)

In [ ]:
#num_samples = 1000
rf_mae = list()
gb_mae = list()
dnn_mae = list()

err_rf_pi = list()
err_gb_pi = list()
err_dnn_pi = list()
err_rf_sv = list() 
err_gb_sv = list()
err_dnn_sv = list()
err_dnn_ig = list()

err_mean = list()
err_median = list()
err_mode = list()
err_box = list()
err_tau = list()
err_major = list()
err_kendall = list()
err_spearman = list()
non_sig_ratio = list()

num_sam = 1000
for i in range(len(_noise)):
    for j in range(len(_informative)):
        for k in range(len(_num_feats)):
            print(f"Noise:{_noise[i]}, n_inform:{_informative[j]}%, n_feats:{_num_feats[k]}, n_samples:1000")
            start = timeit.default_timer()

            # high informative high noise
            _features, _output, _coef = make_regression(
                n_samples=num_sam,
                # three features
                n_features=_num_feats[k],
                # where only two features are useful,
                n_informative=int(_num_feats[k]*(_informative[j]/100)),
                # a single target value per observation
                n_targets=1,
                # 0.0 standard deviation of the guassian noise
                noise=_noise[i],
                # show the true coefficient used to generated the data
                coef=True,
            )

            temp_non_sig_ratio ,temp_rf_mae, temp_gb_mae, temp_dnn_mae, temp_err_rf_pi, temp_err_gb_pi, temp_err_dnn_pi, temp_err_rf_sv, temp_err_gb_sv, temp_err_dnn_sv, temp_err_dnn_ig, temp_err_mean, temp_err_median, temp_err_mode, temp_err_box, temp_err_tau, temp_err_major, temp_err_kendall, temp_err_spearman = ensemble_feature_importance(_num_feats[k], _features, _output, _coef, informative=int((_num_feats[k])*(_informative[j]/100)), noise=_noise[i])

            stop = timeit.default_timer()

            print('Time: ', stop - start)
            non_sig_ratio.append(temp_non_sig_ratio)
            rf_mae.append(temp_rf_mae)
            gb_mae.append(temp_gb_mae)
            dnn_mae.append(temp_dnn_mae)
            err_rf_pi.append(temp_err_rf_pi)
            err_gb_pi.append(temp_err_gb_pi)
            err_dnn_pi.append(temp_err_dnn_pi)
            err_rf_sv.append(temp_err_rf_sv)
            err_gb_sv.append(temp_err_gb_sv)
            err_dnn_sv.append(temp_err_dnn_sv)
            err_dnn_ig.append(temp_err_dnn_ig)
            err_mean.append(temp_err_mean)
            err_median.append(temp_err_median)
            err_mode.append(temp_err_mode)
            err_box.append(temp_err_box)
            err_tau.append(temp_err_tau)
            err_major.append(temp_err_major)
            err_kendall.append(temp_err_kendall)
            err_spearman.append(temp_err_spearman)
            

In [ ]:
d_2 = {'non_sig_ratio':non_sig_ratio,'noise_level':noise_level, 'informative_level':informative_level,'num_features_level':num_features_level,
     'non_sig_ratio':non_sig_ratio,'rf_mae':rf_mae,'gb_mae':gb_mae,'dnn_mae':dnn_mae,'err_rf_pi':err_rf_pi,
     'err_gb_pi':err_gb_pi,'err_dnn_pi':err_dnn_pi,'err_rf_sv':err_rf_sv,'err_gb_sv':err_gb_sv,
     'err_dnn_sv':err_dnn_sv,'err_dnn_ig':err_dnn_ig,'err_mean':err_mean,'err_median':err_median,
     'err_mode':err_mode,'err_box':err_box,'err_tau':err_tau,'err_major':err_major,'err_kendall':err_kendall,
     'err_spearman':err_spearman}

df_1000_samples_rank_avg_2 = pd.DataFrame(data=d_2)
df_1000_samples_rank_avg_2.to_csv('df_1000_samples_rank_avg_2.csv', index=False)

## Design choices

### Feature Importance Selection

### Mean Decrease Impurity
#### “gini importance” or “mean decrease impurity” and is defined as the total decrease in node impurity (weighted by the probability of reaching that node (which is approximated by the proportion of samples reaching that node)) averaged over all trees of the ensemble. Gini Importance or Mean Decrease in Impurity (MDI) calculates each feature importance as the sum over the number of splits (across all tress) that include the feature, proportionally to the number of samples it splits.

#### At each split in each tree, the improvement in the split-criterion is the importance measure attributed to the splitting variable, and is accumulated over all the trees in the forest separately for each variable.

---
### Permutation Importance / Mean Decrease Accuracy
#### It is based on experiments on out-of-bag(OOB) samples, via destroying the predictive power of a feature without changing its marginal distribution.

---
### MDA vs MDI
#### The impurity-based feature importance ranks the numerical features to be the most important features as impurity-based importances are biased towards high cardinality features. It would potentially rank non-predictive numerical feature highly compared to categorical features. Additionally, impurity-based importances are computed on training set statistics and therefore do not reflect the ability of feature to be useful to make predictions that generalize to the test set.
---
### SHAP vs Permutation Importance
#### SHAP feature importance is an alternative to permutation feature importance. There is a big difference between both importance measures: Permutation feature importance is based on the decrease in model performance. SHAP is based on magnitude of feature attributions. SHAP ignores feature dependence. By replacing feature values with values from random instances, it is usually easier to randomly sample from the marginal distribution. However, if features are dependent, e.g. correlated, this leads to putting too much weight on unlikely data points. But SHAP has a solid theoretical foundation in game theory. The prediction is fairly distributed among the feature values. We get contrastive explanations that compare the prediction with the average prediction.

### Feature Importance Ensemble Methods